## Dog Breed Classifier
---

This project is a GUI (Graphical User Interface) for classifying dog breed by images. It uses a pre-trained neural network called ResNet-18 to perform the classification.

- In the develop mode, the code has 3 main functions:

  1. `download_images()`: Download images of dogs from the internet and store them locally, using AZURE BING SEARCH API.

  2. `train_model()`: Trains the dog image classification model using transfer learning on top of the pre-trained ResNet-18.

  3. `classify_image()`: Uses the trained model to classify a dog image.

- In the default mode, the code has only `classify_image()`, and the model is downloaled automatic.

The code also has a **[Gradio Interface](https://huggingface.co/spaces/ericxlima/DogBreedClassifier)** that allows the user to load a dog image and get the model's classification. The model classifies images into 13 different dog breeds:

![img](https://imgur.com/P3HGygv.png)


In [ ]:
# @title Run the Breed Dogs Classifier
# @markdown ### ⬅️ Run this cell
# @markdown ---
# @markdown ### This will install all dependences
# @markdown This will take an additional ~3.5 mins.

# @markdown ---
# @markdown ### Ago, will download the model at [https://huggingface.co/ericxlima/DogsClassifierModel](https://huggingface.co/ericxlima/DogsClassifierModel)
# @markdown ---
# @markdown Do you want execute as a developer?
# @markdown
DEVELOP_MODE = False  # @param {type:"boolean"}
# @markdown - DEVELOP_MODE: This will install all develop requirements and configure (You need set manually the AZURE API KEY)
AZURE_SEARCH_KEY = ""  # @param {type:"string"}

#       Install Main Packages
!pip install 'accelerate==0.18.0'
!pip install 'scipy==1.10.1'
!pip install 'ftfy==6.1.1'
!pip install 'gradio==3.28.1'

if DEVELOP_MODE:
    #       Install Fastbook AI toolkit
    ! [-e / content] & & pip install - Uqq fastbook
    !pip install fastbook
    !pip install - -upgrade fastcore
else:
    #       Download the pkl model
    !git clone https://github.com/ericxlima/dog-breed-classifier
    %mv dog-breed-classifier/dogs.pkl ./

#       Import all libs
import os
from pathlib import Path
import gradio as gr
from fastai.vision.all import *

if DEVELOP_MODE:
    import fastbook
    from fastbook import *
    from fastai.vision.widgets import *
    from fastai.vision.all import *
    import uuid
    import requests
    import imghdr
    from PIL import Image
    import numpy as np

    fastbook.setup_book()


DOGS_NAMES = (
    'Zwergspitz', 'Bouledogue Français', 'Shih Tzu', 'Rottweiler', 'Pug',
    'Golden Retriever', 'Deutscher Schäferhund', 'Yorkshire Terrier',
    'Border Collie', 'Dachshund', 'Poodle', 'Labrador Retriever',
    'Pinscher', 'Golden Retriever',
)
AZURE_KEY = os.environ.get(
    'AZURE_SEARCH_KEY',
    AZURE_SEARCH_KEY,
)
PATH = Path('dogs')
USER_MODE = not DEVELOP_MODE


def download_unique_image(url:str, folder_path:str) -> None:
    try:
        response = requests.get(url, timeout=10)
        content_type = response.headers.get('Content-Type')
        if content_type.startswith('image'):
            image_type = imghdr.what(None, response.content)
            extension = 'jpg' if image_type == 'jpeg' else image_type
            filename = str(uuid.uuid4()) + '.' + extension
            filepath = os.path.join(folder_path, filename)
            with open(filepath, 'wb') as f:
                f.write(response.content)
    except Exception as e:
        print(e)


def remove_corrupted_images(folder_path:str) -> None:
    count_corrupted_images = 0
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        try:
            with Image.open(file_path) as img:
                pass
        except Exception as e:
            print(e)
            os.remove(file_path)
            count_corrupted_images += 1
    print(f'{count_corrupted_images} images removed.')


def download_images_() -> None:
    dogs = {f"{dog_name} Dog":None for dog_name in DOGS_NAMES}
    if DEVELOP_MODE:
        if not PATH.exists():
            PATH.mkdir()
            for dog_name in DOGS_NAMES:
                urls = search_images_bing(AZURE_KEY, dog_name).attrgot('contentUrl')
                dogs[dog_name] = urls
                normalized_dog_name = dog_name.replace(' ', '_').lower()
                dest = os.path.join(PATH, normalized_dog_name)
                if not os.path.exists(dest):
                    os.mkdir(dest)
                download_images(dest, urls=urls)
                remove_corrupted_images(dest)


def train_model():
    dogs_datablock = DataBlock(
        blocks=(ImageBlock, CategoryBlock),
        get_items=get_image_files,
        splitter=RandomSplitter(valid_pct=0.2, seed=42),
        get_y=parent_label,
        item_tfms=[
            Resize(128, ResizeMethod.Squish),
            Resize(128, ResizeMethod.Pad, pad_mode='zeros'),
            RandomResizedCrop(128, min_scale=0.3),
        ]
    )
    dogs_dataloaders = dogs_datablock.dataloaders(PATH)
    learn_ = vision_learner(dogs_dataloaders, resnet18, metrics=error_rate)
    learn_.fine_tune(4)
    learn_.export('dogs.pkl')
    return learn_


def get_model_():
    path = Path()
    model = None

    if USER_MODE:
        model_ = load_learner('dogs.pkl')
    else:
        model_ = train_model()
    return model_


def classify_image(image) -> str:
    learing = get_model_()
    pred, pred_idx, probs = learing.predict(image)
    pred = pred.replace('_', ' ').replace('dog', '').title()
    return f"Prediction: {pred};\n Probability: {probs[pred_idx]:.04f}"


#       Interface
iface = gr.Interface(
    classify_image,
    inputs="image",
    outputs="text",
    title="Classificação de Imagens de Cachorros",
    description="Insira uma imagem para ser classificada"
)


def list_breeds():
    html = "<div class='row'>"
    html += "<div class='column'>"
    html += "<h2>List of breed dogs trained:</h2>"
    html += "<ol>" + "".join([f"<li>{breed}</li>" for breed in DOGS_NAMES]) + "</ol>"
    html += "</div>"
    html += "<div class='column'>"
    html += "<h2>Author:</h2>"
    html += "<a href='https://github.com/ericxlima'><img src='https://avatars.githubusercontent.com/u/58092119?v=4' alt='profile image' style='width:25%' /></a>"
    html += "<h2><a href='https://github.com/ericxlima'>Eric de Lima</a></h2>"
    html += "</div>"
    html += "</div>"
    return html


image = gr.Image(shape=(224, 224))
label = gr.Label(num_top_classes=3)
breeds_list = list_breeds()

demo = gr.Interface(
    fn=classify_image,
    inputs=image,
    outputs=label,
    title="🐶 Dog Breed Classifier",
    interpretation="default",
    description="Upload an image of a dog and the model will predict its breed.",
    article=breeds_list,
    css=".row { display: flex; } .column { flex: 50%; }",
)

demo.launch(share=True, debug=True)
